# Robust Detection of Heart Beats using Dynamic Thresholds and Moving Windows


In [1]:
from os import path, listdir, environ

import pandas as pd
import numpy as np
from tqdm import tqdm

from detector import eval_record

RESULTS_DIR = 'results'
DATASETS_DIR = 'databases'
DATASETS = ['set-p', 'training']

## Evaluation


In [2]:
def evaluate_dataset(dataset):
    records = listdir(path.join(DATASETS_DIR, dataset))
    records = [record[:-4] for record in records if record.endswith('.hea')]
    results = pd.DataFrame(
        columns=['record', 'sensitivity', 'positive predictivity', 'tp', 'fp', 'fn']
    )
    environ['DATASET_DIR'] = path.join(DATASETS_DIR, dataset)

    for record in tqdm(records):
        comparitor = eval_record(record)
        results.loc[len(results)] = (
            record,
            comparitor.sensitivity,
            comparitor.positive_predictivity,
            comparitor.tp,
            comparitor.fp,
            comparitor.fn,
        )
    results.to_csv(path.join(RESULTS_DIR, dataset) + '.csv', index=False)

In [3]:
evaluate_dataset(DATASETS[0])

100%|██████████| 100/100 [06:02<00:00,  3.63s/it]


In [4]:
results = pd.read_csv(path.join('results', DATASETS[0] + '.csv'))
results['f1'] = (
    2
    * results['sensitivity']
    * results['positive predictivity']
    / (results['sensitivity'] + results['positive predictivity'])
)
display(results.sort_values('f1', ascending=False).head(5))
display(results.sort_values('f1', ascending=False).tail(5))

print(
    f'Sensitivity: {np.mean(results["sensitivity"]):.4f} '
    f'+- {np.std(results["sensitivity"]):.4f}'
)
print(
    f'Positive predictivity: {np.mean(results["positive predictivity"]):.4f} '
    f'+- {np.std(results["positive predictivity"]):.4f}'
)

,record,sensitivity,positive predictivity,tp,fp,fn,f1
0,143,1.0,1.0,920,0,0,1.0
21,192,1.0,1.0,721,0,0,1.0
35,147,1.0,1.0,872,0,0,1.0
36,108,1.0,1.0,907,0,0,1.0
38,134,1.0,1.0,757,0,0,1.0


,record,sensitivity,positive predictivity,tp,fp,fn,f1
25,178,0.979328,0.970551,758,23,16,0.974920
12,183,0.978479,0.966006,682,24,15,0.972202
66,113,0.933835,0.996790,621,2,44,0.964286
82,149,0.949106,0.907895,690,70,37,0.928043
53,118,0.769510,0.944321,424,25,127,0.848000


Sensitivity: 0.9917 +- 0.0246
Positive predictivity: 0.9952 +- 0.0118
